In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# LOADING DATA

In [4]:
data=pd.read_excel('./archivos/DataFramePrueba.xlsb')

In [6]:
from pyxlsb import open_workbook
from openpyxl import Workbook

# Nombre del archivo de origen
input_file = "./archivos/DataFramePrueba.xlsb"  
# Nombres de las hojas a extraer
hojas_a_extraer = ["DataFramePrueba", "diccionario"]  
# Abrir el archivo .xlsb
with open_workbook(input_file) as wb:
    for hoja in hojas_a_extraer:
        try:
            # Acceder a la hoja específica
            with wb.get_sheet(hoja) as sheet:
                # Crear un nuevo archivo de Excel
                nuevo_workbook = Workbook()
                nueva_hoja = nuevo_workbook.active
                nueva_hoja.title = hoja

                # Copiar los datos
                for row in sheet.rows():
                    nueva_hoja.append([cell.v for cell in row])

                # Guardar como archivo .xlsx
                output_file = f"{hoja}.xlsx"
                nuevo_workbook.save(output_file)
                print(f"La hoja '{hoja}' se ha guardado en el archivo '{output_file}'.")
        except Exception as e:
            print(f"Error al procesar la hoja '{hoja}': {e}")


La hoja 'DataFramePrueba' se ha guardado en el archivo 'DataFramePrueba.xlsx'.
La hoja 'diccionario' se ha guardado en el archivo 'diccionario.xlsx'.


# GETTING DATA

In [ ]:
df=pd.read_excel('./archivos/archivosSeparados/DataFramePrueba.xlsx')
diccionario=pd.read_excel('./archivos/archivosSeparados/diccionario.xlsx')

In [11]:
df

,IdentificadorCliente,FechaEvento,UsabilidadCupo,CategoriaPrincipalCredito,DiasMaximosMoraCreditosGenerados,NumeroCreditosGPrevius,NumeroCreditosGCanalFPrevius,NumeroCreditosGEstadoActivosPrevius,NumeroCreditosGEstadoPagadosPrevius,NumeroCreditosGCanalVPrevius,...,TipoMunicipioEntregaTC,CanalMunicipioEntregaTC,NumeroIntentosFallidos,CupoAprobado,UsoAppWeb,ScoreCrediticio,Genero,Edad,DiasMora,PerdidaCartera
0,1,2022-09-19T13:25:31.867Z,0.1184320077740548479,NaN,0.0,9.0,9.0,0.0,9.0,0.0,...,PRINCIPAL,Fisico,0.0,3.000000e+10,NaN,865.0,Femenino,37.0,0,0
1,2,2023-08-23T11:33:46.417Z,0.0771590000000000000,hogar-y-muebles,NaN,5.0,4.0,1.0,4.0,1.0,...,INTERMEDIO,Fisico,0.0,2.000000e+10,NaN,726.0,Femenino,38.0,0,0
2,3,2022-10-01T14:59:48.920Z,0.1855666666666666667,NaN,129.0,33.0,33.0,0.0,33.0,0.0,...,PEQUEÃ‘O,Fisico,1.0,3.000000e+10,NaN,0.0,Femenino,35.0,0,0
3,4,2022-09-22T21:25:09.187Z,null,NaN,0.0,3.0,3.0,0.0,3.0,0.0,...,GRANDE,Fisico,0.0,5.000000e+09,NaN,837.0,Masculino,34.0,0,0
4,5,2023-03-19T17:48:52.310Z,0.4752148710773535878,computacion,0.0,4.0,4.0,0.0,4.0,0.0,...,GRANDE,Fisico,1.0,5.000000e+09,App,487.0,Femenino,55.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146934,146935,2023-10-20T16:31:14.983Z,0.2875120000000000000,belleza-y-cuidado-personal,NaN,NaN,NaN,NaN,NaN,NaN,...,VIRTUAL,Virtual,0.0,5.000000e+09,NaN,632.0,Femenino,25.0,0,0
146935,146936,2023-10-20T20:58:15.520Z,null,electrodomesticos,NaN,NaN,NaN,NaN,NaN,NaN,...,VIRTUAL,Virtual,0.0,3.000000e+09,App,579.0,Femenino,35.0,0,0
146936,146937,2023-10-23T22:57:35.243Z,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,INTERMEDIO,Fisico,0.0,3.000000e+09,NaN,578.0,Masculino,20.0,772,1
146937,146938,2023-10-30T15:09:00.610Z,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,INTERMEDIO,Fisico,0.0,1.500000e+09,NaN,0.0,Desconocido,19.0,1614,1


In [12]:
diccionario

,IdentificadorCliente,Numero unico que identifica a cada cliente
0,FechaEvento,Fecha en la cual se hizo la compra en cuestión...
1,UsabilidadCupo,Division entre el valor de la compra y el cupo...
2,CategoriaPrincipalCredito,Categoría mayoritaria de la compra que quiere ...
3,DiasMaximosMoraCreditosGenerados,En la historia que lleva el cliente sacando cr...
4,NumeroCreditosGPrevius,Cantidad de creditos que ha hecho el cliente a...
5,NumeroCreditosGCanalFPrevius,Cantidad de creditos que ha hecho el cliente a...
6,NumeroCreditosGCanalVPrevius,Cantidad de creditos que ha hecho el cliente a...
7,NumeroCreditosGEstadoActivosPrevius,Cantidad de creditos que ha hecho el cliente a...
8,NumeroCreditosGEstadoPagadosPrevius,Cantidad de creditos que ha hecho el cliente a...
9,NumeroCreditosLPrevius,Cantidad de creditos que ha hecho el cliente a...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146939 entries, 0 to 146938
Data columns (total 30 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   IdentificadorCliente                    146939 non-null  int64  
 1   FechaEvento                             146939 non-null  object 
 2   UsabilidadCupo                          146939 non-null  object 
 3   CategoriaPrincipalCredito               114111 non-null  object 
 4   DiasMaximosMoraCreditosGenerados        108968 non-null  float64
 5   NumeroCreditosGPrevius                  115051 non-null  float64
 6   NumeroCreditosGCanalFPrevius            115051 non-null  float64
 7   NumeroCreditosGEstadoActivosPrevius     115051 non-null  float64
 8   NumeroCreditosGEstadoPagadosPrevius     115051 non-null  float64
 9   NumeroCreditosGCanalVPrevius            115051 non-null  float64
 10  NumeroCreditosLPrevius                  1150

In [21]:
nulos_por_columna=df.isnull().sum()
columnas_con_nulos = nulos_por_columna[nulos_por_columna > 0] 
total_columnas_con_nulos = len(columnas_con_nulos)  

print(f"Columnas con valores nulos:\n{columnas_con_nulos}")
print(f"Total de columnas con nulos: {total_columnas_con_nulos}")

Columnas con valores nulos:
CategoriaPrincipalCredito                 32828
DiasMaximosMoraCreditosGenerados          37971
NumeroCreditosGPrevius                    31888
NumeroCreditosGCanalFPrevius              31888
NumeroCreditosGEstadoActivosPrevius       31888
NumeroCreditosGEstadoPagadosPrevius       31888
NumeroCreditosGCanalVPrevius              31888
NumeroCreditosLPrevius                    31888
NumeroCreditosLEstadoActivosPrevius       31888
NumeroCreditosLEstadoPagadosPrevius       31888
FechaVinculacionCliente                    1824
FechaPrimerUso                            35874
FechaUltimoUso                            35874
TotalPagosEfectuadosGlobalmentePrevius    39016
TotalPagosEfectuadosLocalmentePrevius     39016
CodigoMunicipioEntregaTC                   1830
TipoMunicipioEntregaTC                     1830
CanalMunicipioEntregaTC                      16
NumeroIntentosFallidos                      128
CupoAprobado                                660
UsoAppWeb   

In [24]:
df.describe()

,IdentificadorCliente,DiasMaximosMoraCreditosGenerados,NumeroCreditosGPrevius,NumeroCreditosGCanalFPrevius,NumeroCreditosGEstadoActivosPrevius,NumeroCreditosGEstadoPagadosPrevius,NumeroCreditosGCanalVPrevius,NumeroCreditosLPrevius,NumeroCreditosLEstadoActivosPrevius,NumeroCreditosLEstadoPagadosPrevius,...,FechaUltimoUso,TotalPagosEfectuadosGlobalmentePrevius,TotalPagosEfectuadosLocalmentePrevius,CodigoMunicipioEntregaTC,NumeroIntentosFallidos,CupoAprobado,ScoreCrediticio,Edad,DiasMora,PerdidaCartera
count,146939.00000,108968.000000,115051.000000,115051.000000,115051.000000,115051.000000,115051.000000,115051.000000,115051.000000,115051.000000,...,111065.000000,107923.000000,107923.000000,145109.000000,146811.000000,1.462790e+05,146848.000000,145341.000000,146939.000000,146939.000000
mean,73470.00000,77.787626,12.063155,11.622046,0.292592,11.723644,0.441109,0.359754,0.027058,0.247177,...,44838.689425,34.402481,0.813163,124.663384,0.303315,1.395009e+10,529.940101,31.616928,203.896113,0.225509
std,42417.77994,224.206547,17.146242,17.003708,1.086592,17.013161,1.387426,1.136460,0.215295,0.808488,...,317.957633,50.421206,2.630389,210.947443,1.209912,1.151370e+10,254.322400,9.806177,450.909210,0.417918
min,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,36512.000000,0.000000,0.000000,-1.000000,0.000000,1.000000e+09,-9.000000,18.000000,0.000000,0.000000
25%,36735.50000,0.000000,2.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,44717.000000,5.000000,0.000000,-1.000000,0.000000,4.000000e+09,520.000000,24.000000,0.000000,0.000000
50%,73470.00000,0.000000,6.000000,5.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,...,44884.000000,16.000000,0.000000,4.000000,0.000000,1.000000e+10,611.000000,29.000000,0.000000,0.000000
75%,110204.50000,48.000000,15.000000,15.000000,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,...,45041.000000,42.000000,0.000000,139.000000,0.000000,3.000000e+10,687.000000,37.000000,0.000000,0.000000
max,146939.00000,3309.000000,349.000000,349.000000,31.000000,349.000000,58.000000,123.000000,13.000000,29.000000,...,45230.000000,1102.000000,81.000000,1126.000000,130.000000,1.000000e+11,999.000000,99.000000,1800.000000,1.000000


## Descripción superficial de los datos

<div>

<p>
    <ul>
       <li> 146939 filas
       </li> 
       <li> 30 columnas de varios tipos: object,date,float64,int64
       </li>
       <li> 24 columnas con entradas nulas
       </li>
       <li> TotalPagosEfectuadosLocalmentePrevius y  TotalPagosEfectuadosGlobalmentePrevius son las columnas con mas nulos con 39016
       </li>
       <li> Columna a predecir : PerdidaCartera (Si y no)
       </li>    
    </ul>
</p>

</div>


## Análisis de cada columna

<div>

<p>
En esta sección vamos realizar la identificación y conteo de las entidades de cada columna. Esta parte es muy importante a la hora de determinar patrones, realizar segmentaciones y tomar deciciones a la hora de etiquetar, completar datos faltantes o la eliminación de columnas.
</p>
</div>

In [27]:
df.columns

Index(['IdentificadorCliente', 'FechaEvento', 'UsabilidadCupo',
       'CategoriaPrincipalCredito', 'DiasMaximosMoraCreditosGenerados',
       'NumeroCreditosGPrevius', 'NumeroCreditosGCanalFPrevius',
       'NumeroCreditosGEstadoActivosPrevius',
       'NumeroCreditosGEstadoPagadosPrevius', 'NumeroCreditosGCanalVPrevius',
       'NumeroCreditosLPrevius', 'NumeroCreditosLEstadoActivosPrevius',
       'NumeroCreditosLEstadoPagadosPrevius', 'FechaVinculacionCliente',
       'FechaPrimerUso', 'FechaUltimoUso',
       'TotalPagosEfectuadosGlobalmentePrevius',
       'TotalPagosEfectuadosLocalmentePrevius', 'CodigoAlmacenEntregaTC',
       'CodigoMunicipioEntregaTC', 'TipoMunicipioEntregaTC',
       'CanalMunicipioEntregaTC', 'NumeroIntentosFallidos', 'CupoAprobado',
       'UsoAppWeb', 'ScoreCrediticio', 'Genero', 'Edad', 'DiasMora',
       'PerdidaCartera'],
      dtype='object')

In [38]:
df[ 'FechaEvento'].value_counts()

FechaEvento
2023-10-29T19:35:41.283Z    1
2022-09-19T13:25:31.867Z    1
2023-08-23T11:33:46.417Z    1
2022-10-01T14:59:48.920Z    1
2022-09-22T21:25:09.187Z    1
                           ..
2023-03-21T22:16:01.433Z    1
2022-10-01T22:10:17.433Z    1
2023-08-31T18:23:56.993Z    1
2022-05-11T15:22:39.510Z    1
2022-05-31T09:22:36.477Z    1
Name: count, Length: 146939, dtype: int64

### Datos sin nulos en ninguna columna

<div>

<p>
Podemos observar que si quitamos los datos nulos en todas las columnas, nos queda aproximadamente la mitad de los datos. Tenemos que evaluar antes de usar o no estos datos si esos nulos tienen alguna interpretación o simplemente es auscencia de datos y podemos prescindir de estos. Para esto debemos revisar columna por columna y analizarlas con base a la definición brindada en el diccionario.

</p>
</div>

In [23]:
df_sin_nulos = df.dropna()
df_sin_nulos

,IdentificadorCliente,FechaEvento,UsabilidadCupo,CategoriaPrincipalCredito,DiasMaximosMoraCreditosGenerados,NumeroCreditosGPrevius,NumeroCreditosGCanalFPrevius,NumeroCreditosGEstadoActivosPrevius,NumeroCreditosGEstadoPagadosPrevius,NumeroCreditosGCanalVPrevius,...,TipoMunicipioEntregaTC,CanalMunicipioEntregaTC,NumeroIntentosFallidos,CupoAprobado,UsoAppWeb,ScoreCrediticio,Genero,Edad,DiasMora,PerdidaCartera
4,5,2023-03-19T17:48:52.310Z,0.4752148710773535878,computacion,0.0,4.0,4.0,0.0,4.0,0.0,...,GRANDE,Fisico,1.0,5.000000e+09,App,487.0,Femenino,55.0,0,0
10,11,2023-07-12T12:09:24.737Z,0.1248160000000000000,"electronica,-audio-y-video",0.0,7.0,7.0,0.0,7.0,0.0,...,VIRTUAL,Virtual,0.0,5.000000e+09,App,0.0,Masculino,35.0,0,0
23,24,2023-10-25T14:46:57.737Z,0.3108332753020750192,celulares-y-telefonos,0.0,34.0,32.0,0.0,34.0,2.0,...,PRINCIPAL,Fisico,0.0,3.000000e+10,App,752.0,Femenino,29.0,0,0
37,38,2022-10-11T10:46:39.873Z,0.1377377311729431286,belleza-y-cuidado-personal,0.0,26.0,25.0,0.0,26.0,1.0,...,PRINCIPAL,Fisico,0.0,3.000000e+10,App,0.0,Femenino,53.0,0,0
43,44,2023-10-23T12:58:15.907Z,0.0116811037988396770,pines-virtuales,72.0,51.0,45.0,0.0,49.0,6.0,...,PRINCIPAL,Fisico,0.0,3.000000e+10,App,977.0,Femenino,56.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146892,146893,2023-09-19T23:31:40.373Z,0.3950762553867783245,belleza-y-cuidado-personal,0.0,5.0,5.0,5.0,0.0,0.0,...,VIRTUAL,Virtual,0.0,1.000000e+10,App,612.0,Masculino,33.0,515,1
146897,146898,2023-10-02T13:55:53.683Z,0.0511580000000000000,animales-y-mascotas,0.0,2.0,1.0,1.0,1.0,1.0,...,VIRTUAL,Virtual,0.0,1.000000e+10,App,646.0,Femenino,24.0,0,0
146898,146899,2023-10-23T17:15:06.223Z,10000000000000000000,belleza-y-cuidado-personal,0.0,1.0,1.0,0.0,1.0,0.0,...,GRANDE,Fisico,0.0,1.500000e+09,App,0.0,Femenino,20.0,0,0
146913,146914,2023-10-22T19:20:12.033Z,0.1865260000000000000,celulares-y-telefonos,0.0,1.0,1.0,0.0,1.0,0.0,...,VIRTUAL,Virtual,0.0,2.000000e+10,App,762.0,Femenino,41.0,0,0


<div>

<p>
 
 <ul>
 <li>
 </li>
 </ul>

</p>
</div>